In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.svm import SVR
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
import xgboost as xgb
import lightgbm as lgb
from fancyimpute import KNN  
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

C:\Users\colin.huang\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\colin.huang\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [48]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#scaled = scaler.fit_transform(df)dd
#unscaled = scaler.inverse_transform(scaled)

In [96]:
train['new_feature']=train['total_price']/train['building_area']


In [112]:
X=train.iloc[:,1:-2]
#Y=scaler.fit_transform(train.iloc[:,-1].reshape(-1,1))
#Y=np.log(train.iloc[:,-1])
Y=(train.iloc[:,-1])
X_NA=train_NA.iloc[:,1:-1]
Y_NA=train_NA.iloc[:,-1]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_NA, Y_NA, test_size=0.3, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(ans, Y_NA, test_size=0.3, random_state=42)
model=xgb.XGBRegressor(n_estimators=500,max_depth=17,nthread=7,seed=100,learning_rate=0.05,objective='reg:linear',
                                   )
model.fit(X_train,y_train)

ans=model.predict(X_test)

score(ans,np.array(y_test)) #4946.4

array([8401.66745888])

In [113]:
model=xgb.XGBRegressor(n_estimators=700,max_depth=16,nthread=7,seed=100,learning_rate=0.05)

In [114]:
model.fit(X,Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=16, min_child_weight=1, missing=None, n_estimators=700,
       n_jobs=1, nthread=7, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=100,
       silent=True, subsample=1)

# XGBoost

In [158]:
lr=[0.1,0.01,0.001]
max_depth=[10,11,12,13,14,15]
for l in lr: 
    for m in max_depth:     
        scores=[]
        xgb_model=xgb.XGBRegressor(n_estimators=200,max_depth=m,nthread=7,seed=100,learning_rate=l)
        cv=KFold(n_splits=5, random_state=None, shuffle=False)
        k=1
        for train_index, test_index in cv.split(X):
            print("lr=",l,"depth=",m,"now fold:",k,end="  ")
            X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index,:], Y[train_index], Y[test_index]
            xgb_model.fit(X_train, y_train)
            ans=xgb_model.predict(X_test)
            scores.append(score(ans, np.array(y_test)))
            print("score=",scores[k-1])
            k+=1
    #print("scores=",np.mean(scores))

lr= 0.1 depth= 10 now fold: 1  score= 4156.607772312238
lr= 0.1 depth= 10 now fold: 2  score= 4230.753325565148
lr= 0.1 depth= 10 now fold: 3  score= 4210.8002373513855
lr= 0.1 depth= 10 now fold: 4  score= 4175.79584351925
lr= 0.1 depth= 10 now fold: 5  score= 4124.965148975333
lr= 0.1 depth= 11 now fold: 1  score= 4332.445818080548
lr= 0.1 depth= 11 now fold: 2  score= 4385.768677482323
lr= 0.1 depth= 11 now fold: 3  score= 4434.975492535057
lr= 0.1 depth= 11 now fold: 4  score= 4439.139779941395
lr= 0.1 depth= 11 now fold: 5  score= 4409.14220479562
lr= 0.1 depth= 12 now fold: 1  score= 4578.291105219594
lr= 0.1 depth= 12 now fold: 2  score= 4552.448340899642
lr= 0.1 depth= 12 now fold: 3  score= 4563.317247972148
lr= 0.1 depth= 12 now fold: 4  score= 4555.814031931248
lr= 0.1 depth= 12 now fold: 5  score= 4594.983489912719
lr= 0.1 depth= 13 now fold: 1  score= 4735.795812283651
lr= 0.1 depth= 13 now fold: 2  score= 4620.780749743327
lr= 0.1 depth= 13 now fold: 3  score= 4746.656512

KeyboardInterrupt: 

In [30]:
def score(y_predict,y_true):
    score_hit_rate=[]
    score_mape=[]
    for i in range(len(y_predict)):
        ans=(np.abs(y_predict[i]-y_true[i]))/y_true[i]
        if ans>0.1:
            hit_rate=0
        else:
            hit_rate=1
        score_hit_rate.append(hit_rate)
        score_mape.append(ans)
    score=((round(sum(score_hit_rate),4))/len(y_predict)*10000)+(1-((sum(score_mape))/len(y_predict)))
    return(score)

In [115]:
submit_test.iloc[:,-1]=model.predict(test.iloc[:,1:])

In [116]:
#submit_test.iloc[:,-1]=np.expm1(submit_test.iloc[:,-1])*test['building_area']
submit_test.iloc[:,-1]=(submit_test.iloc[:,-1])*test['building_area'] # NO LOG 

In [55]:
submit_test.iloc[:,-1]=scaler.inverse_transform(submit_test.iloc[:,-1])

In [117]:
submit_test.to_csv("C:/Users/colin.huang/Desktop/submit_test3.csv")

In [91]:
train=pd.read_csv("C:/Users/colin.huang/Desktop/train.csv")
#train=pd.read_csv("C:/Users/colin.huang/Desktop/train_drop.csv")
test=pd.read_csv("C:/Users/colin.huang/Desktop/test.csv")
#test=pd.read_csv("C:/Users/colin.huang/Desktop/test_drop.csv")
submit_test=pd.read_csv("C:/Users/colin.huang/Desktop/submit_test.csv")

null_columns_train=train.columns[train.isnull().any()] #check nan columns
train[null_columns_train].isnull().sum() # how many in each columns
null_columns_test=test.columns[test.isnull().any()]
test[null_columns_test].isnull().sum()
train_NA=train.drop(null_columns_train,axis=1)
test_NA=test.drop(null_columns_test,axis=1)

#alldata=pd.concat([train,test],axis=0)

In [53]:
train[null_columns_train].isnull().sum()

parking_area             56897
parking_price            46065
txn_floor                15902
village_income_median     1142
dtype: int64

In [49]:
cor=train.corr()

In [50]:
cor["total_price"].sort_values(ascending=False)


total_price              1.000000
building_area            0.459167
parking_price            0.458674
parking_area             0.377045
land_area                0.313409
master_rate              0.168603
bachelor_rate            0.168585
doc_Rate                 0.165599
XIII_5000                0.161861
jobschool_rate           0.150532
village_income_median    0.145541
VII_5000                 0.140251
V_5000                   0.136286
VII_1000                 0.135433
V_1000                   0.135189
XIII_10000               0.134800
IX_5000                  0.133290
VII_10000                0.133018
XI_10000                 0.131184
V_500                    0.130854
VI_10000                 0.130448
IX_1000                  0.129597
IX_10000                 0.129468
V_10000                  0.128324
VII_500                  0.127891
VIII_5000                0.126393
VIII_10000               0.124815
III_10000                0.124746
VI_5000                  0.124249
X_10000       